## **Import packages and data**

In [2]:
#----------------- Download Data -----------------
import requests
import os
import sys 

#La Manno et al. 2020, Developing Mouse Brain data
from tqdm import tnrange, tqdm_notebook
def download_file(doi,ext):
	url = 'https://api.datacite.org/dois/'+doi+'/media'
	r = requests.get(url).json()
	netcdf_url = r['data'][0]['attributes']['url']
	r = requests.get(netcdf_url,stream=True)
	#Set file name
	fname = doi.split('/')[-1]+ext
	#Download file with progress bar
	if r.status_code == 403:
		print("File Unavailable")
	if 'content-length' not in r.headers:
		print("Did not get file")
	else:
		with open(fname, 'wb') as f:
			total_length = int(r.headers.get('content-length'))
			pbar = tnrange(int(total_length/1024), unit="B")
			for chunk in r.iter_content(chunk_size=1024):
				if chunk:
					pbar.update()
					f.write(chunk)
		return fname



In [2]:
#dev_all_hvg.mtx
download_file('10.22002/D1.2043','.gz')

#dev_all_raw.mtx
download_file('10.22002/D1.2044','.gz')

#lamannometadata.csv
download_file('10.22002/D1.2045','.gz')

os.system("gunzip *.gz")

os.system("mv D1.2043 dev_all_hvg.mtx")
os.system("mv D1.2044 dev_all_raw.mtx")
os.system("mv D1.2045 metadata.csv")


# os.system("pip3 install --quiet torch --no-cache-dir")
# os.system("pip3 install --quiet anndata --no-cache-dir")
# os.system("pip3 install --quiet matplotlib --no-cache-dir")
# os.system("pip3 install --quiet scikit-learn --no-cache-dir")
# os.system("pip3 install --quiet torchsummary --no-cache-dir")
# os.system("pip install --quiet scanpy==1.6.0 --no-cache-dir")
# #pip3 install --quiet umap-learn --no-cache-dir
# os.system("pip3 install --quiet scvi-tools --no-cache-dir")


#os.system("git clone https://github.com/pachterlab/CP_2023.git")


/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`


  0%|          | 0/147369 [00:00<?, ?B/s]

  0%|          | 0/102743 [00:00<?, ?B/s]

  0%|          | 0/831 [00:00<?, ?B/s]

0

In [3]:
os.system("wget --quiet https://storage.googleapis.com/linnarsson-lab-loom/dev_all.loom")

0

In [3]:
import anndata 
import pandas as pd
import numpy as np
import loompy as lp
# import visualizations as vis
# import tools as tl
import random
import scvi
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.neighbors import NeighborhoodComponentsAnalysis, NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
import torch
import time
import scanpy as sc
import seaborn as sns
import umap
from scipy import stats
import scipy.io as sio

Global seed set to 0
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/pytorch_lightning/utilities/warnings.py:54: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  "pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6"
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [4]:
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['axes.linewidth'] = 0.1

state = 42
ndims = 2

data_path = './'

pcs = 50
pcs2 = 100
hpfs = 96
hpfs2 = 30






count_mat = sio.mmread(data_path+'/dev_all_hvg.mtx')
count_mat = count_mat.todense()

print(count_mat.shape)

rawcount_mat = sio.mmread(data_path+'/dev_all_raw.mtx')
rawcount_mat  = rawcount_mat.todense()
print(rawcount_mat.shape)




(292495, 1999)
(292495, 1999)


In [5]:
#Get original hpf factorization
ds = lp.connect("dev_all.loom")
hpf_mat = ds.ca['HPF']
ds.close()
print(hpf_mat.shape)

(292495, 96)


In [6]:
meta = pd.read_csv(data_path+'/metadata.csv',index_col = 0)
meta.head()


,Age,ClusterName
0,e7.0,ParEndo
1,e7.0,ParEndo
2,e7.0,ParEndo
3,e7.0,ParEndo
4,e7.0,ParEndo


In [7]:
#Filter out nan cells from counts

rawcount_mat = rawcount_mat[meta.ClusterName == meta.ClusterName,:]
count_mat = count_mat[meta.ClusterName == meta.ClusterName,:]
hpf_mat = hpf_mat[meta.ClusterName == meta.ClusterName,:]

print(count_mat.shape)
print(rawcount_mat.shape)
print(hpf_mat.shape)

meta = meta[meta.ClusterName == meta.ClusterName]

#Center and scale log-normalized data
scaled_mat = scale(count_mat)

#convert lab1 to ints
dNames = {}
count = 1
for n in np.unique(meta.ClusterName):
        dNames[n] = count
        count = count + 1

meta['ClusterID'] = [dNames[n] for n in list(meta.ClusterName)]
print(meta.head())


(277123, 1999)
(277123, 1999)
(277123, 96)


/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/utils/validation.py:590: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


    Age ClusterName  ClusterID
0  e7.0     ParEndo        628
1  e7.0     ParEndo        628
2  e7.0     ParEndo        628
3  e7.0     ParEndo        628
4  e7.0     ParEndo        628


In [8]:
clusters = np.unique(meta['ClusterName'].values)
map_dict = {}
for i, c in enumerate(clusters):
	map_dict[c] = i
new_labs = [map_dict[c] for c in meta['ClusterName'].values]





adata = anndata.AnnData(count_mat, obs = meta)
adata.X = np.nan_to_num(adata.X)

adata2 = anndata.AnnData(rawcount_mat, obs = meta)
adata2.X = np.nan_to_num(adata2.X)

adata_hpf = anndata.AnnData(hpf_mat, obs = meta)
adata_hpf.X = np.nan_to_num(adata_hpf.X)

/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  # This is added back by InteractiveShellApp.init_path()
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:

## **Reduce Spaces and Predict Cell Labels**

In [9]:

def knn_infer(embd_space, labeled_idx, labeled_lab, unlabeled_idx,n_neighbors=50):
	"""
	Predicts the labels of unlabeled data in the embedded space with KNN.
	Parameters
	----------
	embd_space : ndarray (n_samples, embedding_dim)
		Each sample is described by the features in the embedded space.
		Contains all samples, both labeled and unlabeled.
	labeled_idx : list
		Indices of the labeled samples (used for training the classifier).
	labeled_lab : ndarray (n_labeled_samples)
		Labels of the labeled samples.
	unlabeled_idx : list
		Indices of the unlabeled samples.
	Returns
	-------
	pred_lab : ndarray (n_unlabeled_samples)
		Inferred labels of the unlabeled samples.
	"""

	# obtain labeled data and unlabled data from indices
	labeled_samp = embd_space[labeled_idx, :]
	unlabeled_samp = embd_space[unlabeled_idx, :]

	from sklearn.neighbors import KNeighborsClassifier

	knn = KNeighborsClassifier(n_neighbors=n_neighbors)
	knn.fit(labeled_samp, labeled_lab)

	pred_lab = knn.predict(unlabeled_samp)
	return pred_lab

In [10]:
#Copied from https://github.com/linnarsson-lab/cytograph2/blob/master/cytograph/decomposition/HPF.py
import logging
from typing import Tuple, List, Any

import numpy as np
import scipy.sparse as sparse
from scipy.special import digamma, gammaln, logsumexp
from sklearn.model_selection import train_test_split
from tqdm import trange


def _find_redundant_components(factors: np.ndarray, max_r: float) -> List[int]:
	n_factors = factors.shape[1]
	(row, col) = np.where(np.corrcoef(factors.T) > max_r)
	g = sparse.coo_matrix((np.ones(len(row)), (row, col)), shape=(n_factors, n_factors))
	(n_comps, comps) = sparse.csgraph.connected_components(g)
	non_singleton_comps = np.where(np.bincount(comps) > 1)[0]
	to_randomize: List[int] = []
	for c in non_singleton_comps:
		to_randomize += list(np.where(comps == c)[0][1:])
	return sorted(to_randomize)


def find_redundant_components(beta: np.ndarray, theta: np.ndarray, max_r: float) -> np.ndarray:
	"""
	Figure out which components are redundant (identical to another factor), and
	return them as a sorted ndarray. For each set of redundant factors, all but the
	first element is returned.
	"""
	return np.intersect1d(_find_redundant_components(beta, max_r), _find_redundant_components(theta, max_r))


class HPF:
	"""
	Bayesian Hierarchical Poisson Factorization
	Implementation of https://arxiv.org/pdf/1311.1704.pdf
	"""
	def __init__(
		self,
		k: int,
		*,
		a: float = 0.3,
		b: float = 1,
		c: float = 0.3,
		d: float = 1,
		min_iter: int = 10,
		max_iter: int = 100,
		stop_interval: int = 10,
		epsilon: float = 0.001,
		max_r: float = 0.99,
		compute_X_ppv: bool = True,
		validation_fraction: float = 0) -> None:
		"""
		Args:
			k				Number of components
			a				Hyperparameter a in the paper
			b				Hyperparameter a' in the paper
			c				Hyperparameter c in the paper
			d				Hyperparameter c' in the paper
			max_iter		Maximum number of iterations
			stop_interval	Interval between calculating and reporting the log-likelihood
			epsilon			Fraction improvement required to continue iterating
			max_r			Maximum Pearson's correlation coefficient allowed before a component is considered redundant
			compute_X_ppv	If true, compute the posterior predictive values X_ppv (same shape as X)
		"""
		self.k = k
		self.a = a
		self.b = b
		self.c = c
		self.d = d
		self.min_iter = min_iter
		self.max_iter = max_iter
		self.stop_interval = stop_interval
		self.epsilon = epsilon
		self.max_r = max_r
		self.compute_X_ppv = compute_X_ppv
		self.validation_fraction = validation_fraction

		self.beta: np.ndarray = None
		self.theta: np.ndarray = None
		self.eta: np.ndarray = None
		self.xi: np.ndarray = None
		self.gamma_shape: np.ndarray = None
		self.gamma_rate: np.ndarray = None
		self.lambda_shape: np.ndarray = None
		self.lambda_rate: np.ndarray = None
		self.redundant: np.ndarray = None
		self.validation_data: sparse.coo_matrix = None

		self.X_ppv: np.ndarray = None
		self.log_likelihoods: List[float] = []

		self._tau_rate: np.ndarray = None
		self._tau_shape: np.ndarray = None
		self._lambda_rate: np.ndarray = None
		self._lambda_shape: np.ndarray = None

	def fit(self, X: sparse.coo_matrix) -> Any:
		"""
		Fit an HPF model to the data matrix

		Args:
			X	Data matrix, shape (n_cells, n_genes)

		Remarks:
			After fitting, the factor matrices beta and theta are available as self.theta of shape
			(n_cells, k) and self.beta of shape (k, n_genes)
		"""
		if type(X) is not sparse.coo_matrix:
			raise TypeError("Input matrix must be in sparse.coo_matrix format")

		(beta, theta, eta, xi, gamma_shape, gamma_rate, lambda_shape, lambda_rate) = self._fit(X)

		self.beta = beta
		self.theta = theta
		self.eta = eta
		self.xi = xi
		self.gamma_rate = gamma_rate
		self.gamma_shape = gamma_shape
		self.lambda_rate = lambda_rate
		self.lambda_shape = lambda_shape
		# Identify redundant components
		self.redundant = find_redundant_components(beta, theta, self.max_r)
		# Posterior predictive distribution
		if self.compute_X_ppv:
			self.X_ppv = theta @ beta.T
		return self

	def _fit(self, X: sparse.coo_matrix, beta_precomputed: bool = False) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
		# Create local variables for convenience
		(n_users, n_items) = X.shape
		k = self.k
		(u, i, y) = (X.row, X.col, X.data)  # u and i are indices of the nonzero entries; y are the values of those entries
		(a, b, c, d) = (self.a, self.b, self.c, self.d)
		logging.info(f"nnz={len(u)}")

		# Compute hyperparameters bp and dp
		def mean_var_prior(X: np.ndarray, axis: int) -> float:
			temp = X.sum(axis=axis)
			return np.mean(temp) / np.var(temp)
		bp = b * mean_var_prior(X, axis=1)
		dp = d * mean_var_prior(X, axis=0)

		# Create the validation dataset
		if self.validation_fraction > 0:
			(u, vu, i, vi, y, vy) = train_test_split(u, i, y, train_size=1 - self.validation_fraction)
			self.validation_data = sparse.coo_matrix((vy, (vu, vi)), shape=X.shape)
		else:
			(vu, vi, vy) = (u, i, y)

		# Initialize the variational parameters with priors and some randomness
		kappa_shape = np.full(n_users, b + k * a, dtype='float32')  # This is actually the first variational update, but needed only once
		kappa_rate = np.random.uniform(0.5 * bp, 1.5 * bp, n_users).astype('float32')
		gamma_shape = np.random.uniform(0.5 * a, 1.5 * a, (n_users, k)).astype('float32')
		gamma_rate = np.random.uniform(0.5 * b, 1.5 * b, (n_users, k)).astype('float32')

		if beta_precomputed:
			tau_shape = self._tau_shape
			tau_rate = self._tau_rate
			lambda_shape = self._lambda_shape
			lambda_rate = self._lambda_rate
		else:
			tau_shape = np.full(n_items, d + k * c, dtype='float32')  # This is actually the first variational update, but needed only once
			tau_rate = np.random.uniform(0.5 * dp, 1.5 * dp, n_items).astype('float32')
			lambda_shape = np.random.uniform(0.5 * c, 1.5 * c, (n_items, k)).astype('float32')
			lambda_rate = np.random.uniform(0.5 * d, 1.5 * d, (n_items, k)).astype('float32')

		self.log_likelihoods = []
		with trange(self.max_iter + 1) as t:
			t.set_description(f"HPF.fit(X.shape={X.shape})")
			for n_iter in t:
				# Compute y * phi only for the nonzero values, which are indexed by u and i in the sparse matrix
				# phi is calculated on log scale from expectations of the gammas, hence the digamma and log terms
				# Shape of phi will be (nnz, k)
				# TODO: rewrite in numba so as to calculate each nnz (and sum over k) without materializing the whole phi matrix
				# TODO: maybe parallelize too?
				phi = (digamma(gamma_shape) - np.log(gamma_rate))[u, :] + (digamma(lambda_shape) - np.log(lambda_rate))[i, :]
				# Multiply y by phi normalized (in log space) along the k axis
				y_phi = y[:, None] * np.exp(phi - logsumexp(phi, axis=1)[:, None])

				# Upate the variational parameters corresponding to theta (the users)
				# Sum of y_phi over users, for each k
				y_phi_sum_u = np.zeros((n_users, k))
				for ix in range(k):
					y_phi_sum_u[:, ix] = sparse.coo_matrix((y_phi[:, ix], (u, i)), X.shape).sum(axis=1).A.T[0]
				gamma_shape = a + y_phi_sum_u
				gamma_rate = (kappa_shape / kappa_rate)[:, None] + (lambda_shape / lambda_rate).sum(axis=0)
				kappa_rate = (b / bp) + (gamma_shape / gamma_rate).sum(axis=1)

				if not beta_precomputed:
					# Upate the variational parameters corresponding to beta (the items)
					# Sum of y_phi over items, for each k
					y_phi_sum_i = np.zeros((n_items, k))
					for ix in range(k):
						y_phi_sum_i[:, ix] = sparse.coo_matrix((y_phi[:, ix], (u, i)), X.shape).sum(axis=0).A
					lambda_shape = c + y_phi_sum_i
					lambda_rate = (tau_shape / tau_rate)[:, None] + (gamma_shape / gamma_rate).sum(axis=0)
					tau_rate = (d / dp) + (lambda_shape / lambda_rate).sum(axis=1)

				if n_iter % self.stop_interval == 0:
					# Compute the log likelihood and assess convergence
					# Expectations
					egamma = gamma_shape / gamma_rate
					elambda = lambda_shape / lambda_rate
					# Sum over k for the expectations
					# This is really a dot product but we're only computing it for the nonzeros (indexed by u and i)
					s = (egamma[vu] * elambda[vi]).sum(axis=1)
					# We use gammaln to compute the log factorial, hence the "y + 1"
					log_likelihood = np.sum(vy * np.log(s) - s - gammaln(vy + 1))
					self.log_likelihoods.append(log_likelihood)

					# Check for convergence
					# TODO: allow for small fluctuations
					if len(self.log_likelihoods) > 1:
						prev_ll = self.log_likelihoods[-2]
						diff = (log_likelihood - prev_ll) / abs(prev_ll)
						t.set_postfix(ll=log_likelihood, diff=diff)
						if diff < self.epsilon and n_iter >= self.min_iter:
							break
					else:
						t.set_postfix(ll=log_likelihood)

		# End of the main fitting loop
		if not beta_precomputed:
			# Save these for future use in self.transform()
			self._tau_shape = tau_shape
			self._tau_rate = tau_rate
			self._lambda_shape = lambda_shape
			self._lambda_rate = lambda_rate

		# Compute beta and theta, which are given by the expectations, i.e. shape / rate
		beta = lambda_shape / lambda_rate
		theta = gamma_shape / gamma_rate
		eta = tau_shape / tau_rate
		xi = kappa_shape / kappa_rate
		return (beta, theta, eta, xi, gamma_shape, gamma_rate, lambda_shape, lambda_rate)

	def transform(self, X: sparse.coo_matrix) -> np.ndarray:
		"""
		Transform the data matrix using an already fitted HPF model

		Args:
			X      Data matrix, shape (n_cells, n_genes)

		Returns:
			Factor matrix theta of shape (n_cells, k)
		"""
		if type(X) is not sparse.coo_matrix:
			raise TypeError("Input matrix must be in sparse.coo_matrix format")

		(_, theta, _, _, _, _, _, _) = self._fit(X, beta_precomputed=True)

		return theta

In [13]:
#2D embeddings From HPF 30 --> 2D
from scipy.sparse import coo_matrix

#Get HPF 30D factorization, as in paper
h_reduce = HPF(k=30)
h_reduce.fit(coo_matrix(rawcount_mat))
hpf30 = h_reduce.transform(coo_matrix(rawcount_mat))

HPF.fit(X.shape=(277123, 1999)):  30%|████████████████▋                                       | 30/101 [18:32<43:52, 37.08s/it, diff=0.000598, ll=-8.42e+7]


In [15]:
#Save mat
np.save('hpf30.npy',hpf30)
# hpf30 = np.load('hpf30.npy')

In [16]:
lab1 = list(meta.ClusterName)
lab2 = list(meta.Age)
# lab3 = list(meta.medical_cond_label)
lab4 = list(meta.ClusterID)


allLabs = np.array([lab1])
allLabs2 = np.array([lab1,lab2])

nanLabs = np.array([[np.nan]*len(lab1)])

#Shuffled labels for over-fitting check
shuff_lab1 = random.sample(lab1, len(lab1))  
shuff_lab2 = random.sample(lab2, len(lab2))  
shuff_allLabs = np.array([shuff_lab1,shuff_lab2])

In [22]:
#2D embeddings, from PCA50D --> 2D
ndims = 2
acc_score_2D = []

for i in range(3):
	reducer = umap.UMAP(n_components = ndims)
	tsne = TSNE(n_components = ndims) 


	tsvd = TruncatedSVD(n_components=pcs)
	x_pca = tsvd.fit_transform(scaled_mat)

	pcaUMAP = reducer.fit_transform(x_pca)
	pcaTSNE = tsne.fit_transform(x_pca)

	#Partially labeled UMAP

	labels = np.array([lab4]).copy().astype(np.int8)
	train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False) #0.7 for training fraction
	#Set 30% to no label (nan)
	unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
	labels[:, unlab_inds] = -1

	pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

	preds = knn_infer(pcaUMAPLab, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_2D.append(acc)



	preds = knn_infer(pcaUMAP, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_2D.append(acc)

	preds = knn_infer(pcaTSNE, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_2D.append(acc)

print(acc_score_2D)

/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  F

[0.407219408927433, 0.38957383595751593, 0.43360958418032886, 0.4101663519251356, 0.39545569361415495, 0.4359551102397248, 0.4029132636491574, 0.3923403538737265, 0.43270745877286887]


In [23]:
#2D embeddings From HPF 96 --> 2D
ndims = 2
acc_score_96_2D = []

for i in range(3):
	reducer = umap.UMAP(n_components = ndims)
	tsne = TSNE(n_components = ndims,early_exaggeration = 1.5, perplexity = 150)  #From paper

	x_pca = hpf_mat

	pcaUMAP = reducer.fit_transform(x_pca)
	pcaTSNE = tsne.fit_transform(x_pca)

	#Partially labeled UMAP

	labels = np.array([lab4]).copy().astype(np.int8)
	train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False) #0.7 for training fraction
	#Set 30% to no label (nan)
	unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
	labels[:, unlab_inds] = -1

	pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

	preds = knn_infer(pcaUMAPLab, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_96_2D.append(acc)



	preds = knn_infer(pcaUMAP, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_96_2D.append(acc)

	preds = knn_infer(pcaTSNE, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_96_2D.append(acc)

print(acc_score_96_2D)

/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  F

[0.2771569818492368, 0.26763053754645943, 0.295800906936743, 0.27851618412980983, 0.27044516881773456, 0.295800906936743, 0.275003909210099, 0.270938330707146, 0.29568062354908164]


In [24]:
#2D embeddings From HPF 30 --> 2D
ndims = 2
acc_score_30_2D = []

for i in range(3):
	reducer = umap.UMAP(n_components = ndims) 
	tsne = TSNE(n_components = ndims,early_exaggeration = 1.5, perplexity = 150) #from paper

	x_pca = hpf30

	pcaUMAP = reducer.fit_transform(x_pca)
	pcaTSNE = tsne.fit_transform(x_pca)

	#Partially labeled UMAP

	labels = np.array([lab4]).copy().astype(np.int8)
	train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False) #0.7 for training fraction
	#Set 30% to no label (nan)
	unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
	labels[:, unlab_inds] = -1

	pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

	preds = knn_infer(pcaUMAPLab, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_30_2D.append(acc)



	preds = knn_infer(pcaUMAP, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_30_2D.append(acc)

	preds = knn_infer(pcaTSNE, train_inds, adata.obs.ClusterID.values[train_inds], unlab_inds)
	acc = accuracy_score(adata.obs.ClusterID.values[unlab_inds], preds)
	acc_score_30_2D.append(acc)

print(acc_score_30_2D)

/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/home/tchari/miniconda3/envs/py37/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  F

[0.234492464245763, 0.23373467890349664, 0.2516929886813332, 0.23533444795939232, 0.22579597531784884, 0.2505984098536151, 0.23752360561482855, 0.2300058938859954, 0.2524387456848335]


In [25]:
# #PCA 50D accuracy
acc_scorePCA = []

for i in range(3):

	tsvd = TruncatedSVD(n_components=pcs)
	x_pca = tsvd.fit_transform(scaled_mat)

	labels = np.array([lab1])
	train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
	unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
	labels[:, unlab_inds] = np.nan

	unlabeled_idx = []
	for i in range(len(adata)):
		if i not in train_inds:
			unlabeled_idx.append(i)

	preds = knn_infer(x_pca, train_inds, adata.obs.ClusterName.values[train_inds], unlabeled_idx)
	acc = accuracy_score(adata.obs.ClusterName.values[unlabeled_idx], preds)
	acc_scorePCA.append(acc)

print(acc_scorePCA)

#Larger PCA reduction (100D)
acc_scorePCA2 = []

for i in range(3):

	tsvd = TruncatedSVD(n_components=pcs2)
	x_pca = tsvd.fit_transform(scaled_mat)

	labels = np.array([lab1])
	train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
	unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
	labels[:, unlab_inds] = np.nan

	unlabeled_idx = []
	for i in range(len(adata)):
		if i not in train_inds:
			unlabeled_idx.append(i)

	preds = knn_infer(x_pca, train_inds, adata.obs.ClusterName.values[train_inds], unlabeled_idx)
	acc = accuracy_score(adata.obs.ClusterName.values[unlabeled_idx], preds)
	acc_scorePCA2.append(acc)

print(acc_scorePCA2)

[0.5040595643335699, 0.5031814956036422, 0.5024838519552065]
[0.5412872728147515, 0.5413714711861145, 0.5385327832373071]


In [26]:
#HPF 96
acc_score_hpf96 = []

for i in range(3):

	x_pca = hpf_mat

	labels = np.array([lab1])
	train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
	unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
	labels[:, unlab_inds] = np.nan

	unlabeled_idx = []
	for i in range(len(adata)):
		if i not in train_inds:
			unlabeled_idx.append(i)

	preds = knn_infer(x_pca, train_inds, adata.obs.ClusterName.values[train_inds], unlabeled_idx)
	acc = accuracy_score(adata.obs.ClusterName.values[unlabeled_idx], preds)
	acc_score_hpf96.append(acc)

print(acc_score_hpf96)

[0.3811179138049244, 0.386963686445265, 0.383884431721135]


In [27]:
#HPF 30
acc_score_hpf30 = []

for i in range(3):

	x_pca = hpf30

	labels = np.array([lab1])
	train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
	unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
	labels[:, unlab_inds] = np.nan

	unlabeled_idx = []
	for i in range(len(adata)):
		if i not in train_inds:
			unlabeled_idx.append(i)

	preds = knn_infer(x_pca, train_inds, adata.obs.ClusterName.values[train_inds], unlabeled_idx)
	acc = accuracy_score(adata.obs.ClusterName.values[unlabeled_idx], preds)
	acc_score_hpf30.append(acc)

print(acc_score_hpf30)

[0.3219866004306145, 0.3208799932641303, 0.3219986287693807]


## **Save Results**

In [28]:
#---------------- Save knn prediction accuracy scores for cell type labels ----------------
vals = pd.DataFrame()

vals['Accuracy'] = acc_scorePCA  + acc_scorePCA2 + acc_score_2D +  acc_score_96_2D + acc_score_30_2D + acc_score_hpf96 + acc_score_hpf30 #acc_score  +  acc_score_scanvi + acc_scoreR + acc_scoreBoth + 

r = 3
vals['Embed'] = ['PCA 50D']*r +['PCA 100D']*r + ['PCA UMAP Sup.','PCA UMAP','PCA t-SNE']*r + ['HPF96 UMAP Sup.','HPF96 UMAP','HPF96 t-SNE']*r +['HPF30 UMAP Sup.','HPF30 UMAP','HPF30 t-SNE']*r + ['HPF 96D']*r + ['HPF 30D']*r #  ['LDVAE']*r + ['SCANVI']*r + ['Recon MCML']*r + ['NCA-Recon MCML']*r +


# vals['Label'] = ['CellType1']*15 #+ ['Gender2']*12 + ['CellType2']*1 #+  ['CellType1'] #+  ['Gender2']
vals.to_csv('allLaMannoPreds052323.csv')  #allLaMannoPreds103122.csv
print('Saved outputs')

Saved outputs
